# Extract cancer variant reads for alignment visualization

## In this notebook, we use a single RNA-seq fastq file from a melanoma cancer cell line from the the CCLE project. Learn more about the project here:
- sequencing data (ENA): https://www.ebi.ac.uk/ena/browser/view/PRJNA523380
- paper: https://www.nature.com/articles/s41586-019-1186-3

### Requirements: kb, samtools, and bowtie2

In [1]:
try:
    import varseek as vk
except ImportError:
    print("varseek not found, installing...")
    !pip install -U -q varseek
try:
    import RLSRWP_2025
except ImportError:
    print("RLSRWP_2025 not found, installing...")
    !pip install -q git+https://github.com/pachterlab/RLSRWP_2025.git

In [ ]:
import anndata
import os
import numpy as np
import pandas as pd
import pyfastx
import glob
# pd.set_option('display.max_columns', None)
import varseek as vk
from varseek.utils import make_bus_df

RLSRWP_2025_dir = os.path.dirname(os.path.abspath(""))  # if this notebook resides in RLSRWP_2025/notebooks/0_data_download.ipynb, then this retrieves RLSRWP_2025
threads = "2"
use_head = True

### File path definitions and imports

In [2]:
vk_count_out_dir = os.path.join(RLSRWP_2025_dir, "data", "varseek_count_out_alignment_visualization")
kb_count_out_dir = os.path.join(vk_count_out_dir, "kb_count_out_vcrs")
adata_path = os.path.join(kb_count_out_dir, "counts_unfiltered", "adata.h5ad")
aligned_reads_parent_dir = os.path.join(vk_count_out_dir, "pseudoaligned_reads_to_vcrs_reference")
bowtie_read_alignments = os.path.join(vk_count_out_dir, "bowtie_read_alignments")

# vk ref out directory and files - downloaded if not already present
vk_ref_out_dir = os.path.join(RLSRWP_2025_dir, "data", "vk_ref_out")
vcrs_index = os.path.join(vk_ref_out_dir, "vcrs_index.idx")
vcrs_t2g = os.path.join(vk_ref_out_dir, "vcrs_t2g_filtered.txt")
vcrs_fasta = os.path.join(vk_ref_out_dir, "vcrs_filtered.fa")

# fastq directories - fastqs_dir downloaded if not already present, and fastqs_processed_dir created with fastp if not already present
fastqs_dir = os.path.join(RLSRWP_2025_dir, "data", "ccle_data_base", "RNASeq_MELHO_SKIN")
fastq_file = os.path.join(fastqs_dir, "SRR8615233_1.fastq.gz")
technology = "bulk"

reference_genome_dir = os.path.join(RLSRWP_2025_dir, "data", "reference", "ensembl_grch37_release93")

# kb count to reference genome directory and files - created if not already present - only used if qc_against_gene_matrix=True
qc_against_gene_matrix = False
kb_count_reference_genome_dir = os.path.join(RLSRWP_2025_dir, "data", "kb_count_reference_genome")
reference_genome_index = os.path.join(reference_genome_dir, "index.idx")  # either already exists or will be created
reference_genome_t2g = os.path.join(reference_genome_dir, "t2g.txt")  # either already exists or will be created
reference_genome_fasta = os.path.join(reference_genome_dir, "Homo_sapiens.GRCh37.dna.primary_assembly.fa")  # if reference_genome_index/reference_genome_t2g do not exist, then I need to supply the reference genome fasta and gtf
reference_genome_gtf = os.path.join(reference_genome_dir, "Homo_sapiens.GRCh37.87.gtf")  # if reference_genome_index/reference_genome_t2g do not exist, then I need to supply the reference genome fasta and gtf

# for bowtie2
reference_cdna_fasta = os.path.join(reference_genome_dir, "Homo_sapiens.GRCh37.cdna.all.fa")
bowtie_reference_dir = os.path.join(reference_genome_dir, "bowtie_index_transcriptome")
bowtie_reference_prefix = os.path.join(bowtie_reference_dir, "index")

# general
w = "47"  # used during creation of the index, so cannot be altered
k = "51"  # used during creation of the index, so cannot be altered
strand = "unstranded"
parity = "single"  # although the original data is paired, we will only be using a single file, so we will run in single-end mode

# software
bustools = "/Users/joeyrich/miniconda3/envs/RLSRWP_2025/lib/python3.10/site-packages/kb_python/bins/darwin/m1/bustools/bustools"
bowtie2 = "bowtie2"
bowtie2_build = "bowtie2-build"
samtools = "samtools"

### Download VCRS reference files with varseek ref

In [6]:
if not os.path.exists(vcrs_index) or not os.path.exists(vcrs_t2g) or not os.path.exists(vcrs_fasta):
    vk.ref(variants="cosmic_cmc", sequences="cdna", w=w, k=k, dlist_reference_source="t2t", index_out=vcrs_index, t2g_out=vcrs_t2g, fasta_out=vcrs_fasta)

### Make bowtie2 index files

In [ ]:
if not os.path.exists(bowtie_reference_dir) or len(os.listdir(bowtie_reference_dir)) == 0:
    os.makedirs(bowtie_reference_dir, exist_ok=True)
    !{bowtie2_build} --threads {threads} {reference_cdna_fasta} {bowtie_reference_prefix}

### Download fastq file

In [ ]:
if not os.path.isfile(fastq_file):
    fastq_file_link = "ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR861/003/SRR8615233/SRR8615233_1.fastq.gz"  # ["ftp.sra.ebi.ac.uk/vol1/fastq/SRR861/003/SRR8615233/SRR8615233_1.fastq.gz", "ftp.sra.ebi.ac.uk/vol1/fastq/SRR861/003/SRR8615233/SRR8615233_2.fastq.gz"]
    os.makedirs(fastqs_dir, exist_ok=True)
    !wget -c --tries=5 --retry-connrefused -O {fastq_file} {fastq_file_link}

In [ ]:
if use_head:
    fastq_file_head = fastq_file.replace(".fastq", "_head.fastq")
    !zcat {fastq_file} | head -1000000 > $fastq_file_head
    fastq_file = fastq_file_head

In [23]:
!head {fastq_file}

@SRR8615233.1 1/1
GAGAAGTTGATGCTGTCACTCATCTTTCCTCCAATCTTTCCCCTATGCCTGGTTGTGGTATTAAGTTACATGCAGACAACAGGGGCCAGAAGATGAACAAT
+
CCCFFFDFHHHHHJJJJJJJJIJJJJJJJJJJJJJIJJJJJJJJJJJJJJJJHIJJIICFGHJIIJJJJIJJJJGGGHHHHFFFDDBBDDDDDCCDDDDC:
@SRR8615233.2 2/1
CAGAGCTGCAGGCACAGCCAAGAGGGCTGAAGAAATGGTAGAACGGAGCAGCTGGTGATGTGGAGACCGGCCCCAGGCTCCTGTCTCCCCCCAGGTGTGTG
+
CCCFFFFFHHHHHIJJJIJJJJJJJJJIJJJJJIJJJJHHIIIJIIJJJJJJJJIGHIJHGHHHHFFFDDDDDDD?@?ABDDDCCDDCCDDDBBB######
@SRR8615233.3 3/1
TCGGCTTTATAGGCCACCGCAAGCAGCAGGGCTCCAGATGACATCACAGGGAAGATCAAGAGGGTGTGGAGGGGCATCGAAGCCTCTCCAGGAGACAGGAG


### Make pyfastx index files

In [7]:
fastq_indexed = pyfastx.Fastq(fastq_file, build_index=True)
vcrs_fasta_indexed = pyfastx.Fasta(vcrs_fasta, build_index=True)

### Make kb reference files if qc_against_gene_matrix=True

In [31]:
kb_count_reference_genome_adata = os.path.join(kb_count_reference_genome_dir, "counts_unfiltered", "adata.h5ad")
if qc_against_gene_matrix and not os.path.exists(kb_count_reference_genome_adata):  # check if kb count was run
    os.makedirs(kb_count_reference_genome_dir, exist_ok=True)
    if not os.path.exists(reference_genome_index) or not os.path.exists(reference_genome_t2g):  # check if kb ref was run
        if not os.path.exists(reference_genome_fasta) or not os.path.exists(reference_genome_gtf):
            reference_genome_out_dir = os.path.dirname(reference_genome_fasta) if reference_genome_fasta else "."
            # using grch37, ensembl 93 to agree with COSMIC
            !gget ref -w dna,gtf -r 93 --out_dir {reference_genome_out_dir} -d human_grch37 && gunzip {reference_genome_fasta}.gz && gunzip {reference_genome_gtf}.gz
        reference_genome_f1 = os.path.join(kb_count_reference_genome_dir, "f1.fa")
        !kb ref -t {threads} -i {reference_genome_index} -g {reference_genome_t2g} -f1 {reference_genome_f1} {reference_genome_fasta} {reference_genome_gtf}

### Perform variant screening with varseek count

In [5]:
if not os.path.exists(adata_path):
    vk_count_output_dict = vk.count(
        fastqs_dir,
        index=vcrs_index,
        t2g=vcrs_t2g,
        technology=technology,
        out=vk_count_out_dir,
        k=k,
        strand=strand,
        parity=parity,
        threads=threads,
        disable_fastqpp=True,
        disable_clean=True,
        disable_summarize=True
        # quality_control_fastqs=True, cut_front=True, cut_tail=True  # equivalent to the fastp step above
        # qc_against_gene_matrix=qc_against_gene_matrix, reference_genome_index=reference_genome_index, reference_genome_t2g=reference_genome_t2g,
    )

16:24:37 - INFO - Setting length_required to 51 if fastqpp is run
16:24:37 - INFO - Skipping vk fastqpp because disable_fastqpp=True
16:24:37 - INFO - Running kb count with command: kb count -t 2 -k 51 -i /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_ref_out/vcrs_index.idx -g /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_ref_out/vcrs_t2g_filtered.txt -x BULK --h5ad -o /Users/joeyrich/Desktop/local/RLSRWP_2025/data/varseek_count_out_alignment_visualization/kb_count_out_vcrs --overwrite --strand unstranded --parity single --num /Users/joeyrich/Desktop/local/RLSRWP_2025/data/ccle_data_base/RNASeq_MELHO_SKIN/head.fastq.gz
[2025-03-17 16:24:42,788]    INFO [count] Using index /Users/joeyrich/Desktop/local/RLSRWP_2025/data/vk_ref_out/vcrs_index.idx to generate BUS file to /Users/joeyrich/Desktop/local/RLSRWP_2025/data/varseek_count_out_alignment_visualization/kb_count_out_vcrs from
[2025-03-17 16:24:42,789]    INFO [count]         /Users/joeyrich/Desktop/local/RLSRWP_2025/data/ccle_d

### Load in adata object

In [14]:
adata = anndata.read_h5ad(adata_path)
adata

AnnData object with n_obs × n_vars = 1 × 5329695

### View top 10 variants by total reads aligned

In [15]:
top10 = set(adata.var.index[np.argsort(np.array(adata.X.todense())[0])][::-1][:10])
top10

{'ENST00000361390:c.910T>C',
 'ENST00000361453:c.142dup',
 'ENST00000361624:c.226G>A',
 'ENST00000361624:c.795dup',
 'ENST00000361624:c.922G>A',
 'ENST00000361899:c.268C>T',
 'ENST00000361899:c.296C>T',
 'ENST00000361899:c.326G>A',
 'ENST00000361899:c.328G>A',
 'ENST00000361899:c.41T>C'}

### Map the reads to the VCRS to which they aligned

In [16]:
bus_df = make_bus_df(kb_count_out_dir, fastq_file, technology=technology, parity=parity, bustools=bustools)
filtered_bus_df = bus_df[bus_df["gene_names"].apply(lambda x: len(x) == 1)]  # remove multi-mapping reads
filtered_bus_df["gene_names_str"] = filtered_bus_df["gene_names"].apply(lambda x: x[0])  # cast to string
filtered_bus_df = filtered_bus_df[(filtered_bus_df["gene_names_str"].isin(top10)) & (filtered_bus_df["counted_in_count_matrix"])]
print(f"Number of reads in filtered bus file: {len(filtered_bus_df)}")
filtered_bus_df.head()

loading in transcripts
loading in barcodes


Processing FASTQ headers: 250000it [00:00, 2373691.85it/s]

loading in ec matrix
loading in bus df
Merging fastq header df and ec_df into bus df


Determining what counts in count matrix


100%|██████████| 1727/1727 [00:00<00:00, 262381.39it/s]


Saving bus df as parquet
Number of reads in filtered bus file: 1486


,barcode,UMI,EC,read_index,fastq_header,transcript_names,file_index,gene_names,counted_in_count_matrix,gene_names_str
7,AAAAAAAAAAAAAAAA,T,5,93111,SRR8615233.93112,[ENST00000361624:c.226G>A],0,[ENST00000361624:c.226G>A],True,ENST00000361624:c.226G>A
8,AAAAAAAAAAAAAAAA,T,5,93194,SRR8615233.93195,[ENST00000361624:c.226G>A],0,[ENST00000361624:c.226G>A],True,ENST00000361624:c.226G>A
9,AAAAAAAAAAAAAAAA,T,5,94027,SRR8615233.94028,[ENST00000361624:c.226G>A],0,[ENST00000361624:c.226G>A],True,ENST00000361624:c.226G>A
10,AAAAAAAAAAAAAAAA,T,6,94594,SRR8615233.94595,[ENST00000361624:c.795dup],0,[ENST00000361624:c.795dup],True,ENST00000361624:c.795dup
11,AAAAAAAAAAAAAAAA,T,6,94595,SRR8615233.94596,[ENST00000361624:c.795dup],0,[ENST00000361624:c.795dup],True,ENST00000361624:c.795dup


In [17]:
# make IDs since the gene names in HGVSC format do not make for good folder names or bowtie2 header names
vcrs_header_to_id = {vcrs: f"vcrs_{i}" for i, vcrs in enumerate(list(top10))}
filtered_bus_df["vcrs_ids"] = filtered_bus_df["gene_names_str"].map(vcrs_header_to_id)

vcrs_header_to_id_file = os.path.join(vk_count_out_dir, "vcrs_header_to_id.txt")
with open(vcrs_header_to_id_file, "w") as f:
    for key, value in vcrs_header_to_id.items():
        f.write(f"{key}\t{value}\n")  # Tab-separated

id_to_vcrs_header = {v: k for k, v in vcrs_header_to_id.items()}
vcrs_header_to_id

{'ENST00000361899:c.268C>T': 'vcrs_0',
 'ENST00000361453:c.142dup': 'vcrs_1',
 'ENST00000361899:c.41T>C': 'vcrs_2',
 'ENST00000361624:c.795dup': 'vcrs_3',
 'ENST00000361624:c.922G>A': 'vcrs_4',
 'ENST00000361899:c.326G>A': 'vcrs_5',
 'ENST00000361390:c.910T>C': 'vcrs_6',
 'ENST00000361899:c.296C>T': 'vcrs_7',
 'ENST00000361624:c.226G>A': 'vcrs_8',
 'ENST00000361899:c.328G>A': 'vcrs_9'}

In [48]:
for vcrs_id in filtered_bus_df["vcrs_ids"].unique():  # Get unique gene names
    temp_df = filtered_bus_df[filtered_bus_df["vcrs_ids"] == vcrs_id]  # Filter
    fastq_headers = temp_df["fastq_header"].tolist()  # Get values as a list

    gene_dir = os.path.join(aligned_reads_parent_dir, vcrs_id)
    os.makedirs(gene_dir, exist_ok=True)
    
    aligned_reads_file = os.path.join(gene_dir, "1.fastq")
    with open(aligned_reads_file, "w") as f:
        for header in fastq_headers:
            sequence = fastq_indexed[header].seq
            qualities = fastq_indexed[header].qual
            f.write(f"@{header}\n{sequence}\n+\n{qualities}\n")

### Align pulled out reads to the human transcriptome and the VCRSs to generate bam files:

In [57]:
for folder in glob.glob(aligned_reads_parent_dir + "/*/"):
    variant = folder.split("/")[-2]
    variant_header = id_to_vcrs_header[variant]
    print(f"{variant} ({variant_header})\n")

    outfolder = f"{bowtie_read_alignments}/{variant}"
    os.makedirs(outfolder, exist_ok=True)
    
    # Align reads to human ref using bowtie2
    variant_sam = variant + "_reads.sam"
    !$bowtie2 \
        --very-sensitive \
        -k 1 \
        -x $bowtie_reference_prefix \
        -p $threads \
        -q $aligned_reads_parent_dir/$variant/1.fastq \
        -S $outfolder/$variant_sam

    # Convert sam to bam
    variant_bam = variant + "_reads.bam"
    !$samtools view \
        -bS -F4 $outfolder/$variant_sam \
        > $outfolder/$variant_bam
    
    # Sort bam file
    variant_bam_sorted_prefix = "final_" + variant + "_sorted_reads"
    variant_bam_sorted = variant_bam_sorted_prefix + ".bam"
    !$samtools sort \
        $outfolder/$variant_bam \
        $outfolder/$variant_bam_sorted_prefix
    
    # Create an index for the sorted bam file (creates a .bai file)
    !$samtools index $outfolder/$variant_bam_sorted


    # repeat but use vcrs as the input instead of reads
    variant_reference_sequence = vcrs_fasta_indexed[variant_header].seq.strip()

    # Align vcrs to human ref using bowtie2
    variant_sam = variant + "_vcrs.sam"
    !$bowtie2 \
        --very-sensitive \
        -k 1 \
        -x $bowtie_reference_prefix \
        -p $threads \
        -c $variant_reference_sequence \
        -S $outfolder/$variant_sam

    # Convert sam to bam
    variant_bam = variant + "_vcrs.bam"
    !$samtools view \
        -bS -F4 $outfolder/$variant_sam \
        > $outfolder/$variant_bam

    # Sort bam file
    variant_bam_sorted_prefix = "final_" + variant + "_sorted_vcrs"
    variant_bam_sorted = variant_bam_sorted_prefix + ".bam"
    !$samtools sort \
        $outfolder/$variant_bam \
        $outfolder/$variant_bam_sorted_prefix

    # Create an index for the sorted bam file (creates a .bai file)
    !$samtools index $outfolder/$variant_bam_sorted

vcrs_4 (ENST00000361624:c.922G>A)

69 reads; of these:
  69 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    69 (100.00%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
100.00% overall alignment rate
[samopen] SAM header is present: 180253 sequences.
1 reads; of these:
  1 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    1 (100.00%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
100.00% overall alignment rate
[samopen] SAM header is present: 180253 sequences.
vcrs_3 (ENST00000361624:c.795dup)

10 reads; of these:
  10 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    10 (100.00%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
100.00% overall alignment rate
[samopen] SAM header is present: 180253 sequences.
1 reads; of these:
  1 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    1 (100.00%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
100.00% overall alignment rate
[samopen] SAM h

### Load the files in bowtie_read_alignments into NCBI Genome workbench (or another genome viewer) to visualize the alignments (the ones with "final_")